https://www.drugs.com/alpha/a.html

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

# params = {
#     'ie': 'UTF-8',
#     'wd': 'Python'
# }


In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问服务器，获取cookie
    res = requests.get('https://www.drugs.com/alpha/a.html', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，s
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

# 1、先获取文章链接列表

In [19]:
# 不应该把mislist作为参数传进去
def get_term_url(url, tag, letter, subletter, headers_pc, randomproxy, cookie_dict, mislist):
    term_results = []
    # pbar.set_description(letter)
    try:
        requests.adapters.DEFAULT_RETRIES = 20 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/9_drugs/{tag}_{letter}{subletter}列表页.html"):   
#             return None # 这里return None会导致task_monitor那里接收空值而报错
        with open(f'D:/Intern_Project/9_drugs/{tag}_{letter}{subletter}列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)
        size = round(os.path.getsize(f'D:/Intern_Project/9_drugs/{tag}_{letter}{subletter}列表页.html') / float(1024),2)
        if  size<2.0:
            time.sleep(5)
            headers_pc, randomproxy, cookie_dict = changeparams()
            
            r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
            r.encoding = 'utf-8'
            html = r.text
            with open(f'D:/Intern_Project/9_drugs/{tag}_{letter}{subletter}列表页.html', 'w+', encoding='utf-8') as f:
                f.write(html)
            size2 = round(os.path.getsize(f'D:/Intern_Project/9_drugs/{tag}_{letter}{subletter}列表页.html') / float(1024),2)
            if  size2<2.0:
                # 将失败的单独记录下来，之后再调用一次获取
                mis = f'{tag}_{letter}{subletter}'
                mislist.append(mis)
                print("重定向")

        selector = Selector(text=html)
        termUrl_list = selector.xpath(
            "//ul[@class='ddc-list-unstyled']/li/a")
        #print(len(termUrl_list))
        for termUrl in termUrl_list:
            url = "https://www.drugs.com" + termUrl.xpath('./@href').extract()[0]
            term = termUrl.xpath('./text()').extract()[0]
            # print(term)
            tmp = {
                'term': term,
                'url': url
            }
            term_results.append(tmp)
            # time.sleep()
        # pbar.update(1)
    except Exception as e:
        print(e)
        time.sleep(5)
        headers_pc, randomproxy, cookie_dict = changeparams()
        
        mis = f'{tag}_{letter}{subletter}'
        mislist.append(mis)
        
    return term_results, mislist

In [20]:
def task_monitor(task_list, mt_pool, well_results, mis_results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                well_result, mis_result = future.result()
                well_results.extend(well_result)
                mis_results.extend(mis_result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [21]:
letters = [chr(i) for i in range(97, 123)] # 生成26个英文字母的列表
subletters = [chr(i) for i in range(97, 123)] # 生成26个英文字母的列表

letters.append("0-9")
subletters.append("0-9")
# pbar = tqdm(total=len(letters))
tags = ['patient', 'professional']

all_wellresults = []
mislist = []

for tag in tags:
    all_craw_tasks = []
    task_wellresults = []
    task_mislist = []
    print(tag)
    with ThreadPoolExecutor(max_workers=10) as mt_pool:
        for letter in letters:
            headers_pc, randomproxy, cookie_dict = changeparams()
            for subletter in subletters:
                if tag == 'patient':
                    tag_url = f'https://www.drugs.com/alpha/{letter}{subletter}.html'
                if tag == 'professional':
                    tag_url = f'https://www.drugs.com/alpha/{letter}{subletter}.html?pro=1'
                tmp_task = mt_pool.submit(get_term_url, tag_url, tag, letter, subletter, headers_pc, randomproxy, cookie_dict, mislist)
                all_craw_tasks.append(tmp_task)
            time.sleep(2)
            
        task_monitor(all_craw_tasks, mt_pool, task_wellresults, task_mislist)
    
    all_wellresults.extend(task_wellresults)
    mislist.extend(task_mislist)
    with open(f'D:/Intern_Project/9_drugs/术语链接列表.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(all_wellresults, indent=2, ensure_ascii=False))

patient
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向重定向

重定向
重定向
重定向
重定向
重定向
重定向
重定向
professional
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向
重定向


In [31]:
print(len(mislist))
print(mislist[:20])

10672600
['patient_nq', 'patient_nn', 'patient_nt', 'patient_nu', 'patient_nv', 'patient_ns', 'patient_nx', 'patient_np', 'patient_nr', 'patient_nw', 'patient_ny', 'patient_n0-9', 'patient_nz', 'patient_oa', 'patient_oc', 'patient_ob', 'patient_od', 'patient_oe', 'patient_of', 'patient_og', 'patient_nq', 'patient_nn', 'patient_nt', 'patient_nu', 'patient_nv', 'patient_ns', 'patient_nx', 'patient_np', 'patient_nr', 'patient_nw', 'patient_ny', 'patient_n0-9', 'patient_nz', 'patient_oa', 'patient_oc', 'patient_ob', 'patient_od', 'patient_oe', 'patient_of', 'patient_og', 'patient_nq']


In [37]:
mislist = ['patient_nq', 'patient_nn', 'patient_nt', 'patient_nu', 'patient_nv', 'patient_ns', 'patient_nx', 'patient_np', 'patient_nr', 'patient_nw', 'patient_ny', 'patient_n0-9', 'patient_nz', 
           'patient_oa', 'patient_oc', 'patient_ob', 'patient_od', 'patient_oe', 'patient_of', 'patient_og', 
           'professional_ix', 'professional_iy', 'professional_iz', 'professional_i0-9', 
           'professional_ja', 'professional_jb', 'professional_jc', 'professional_jd', 'professional_je', 'professional_jf', 'professional_jg', 'professional_jh', 'professional_ji', 'professional_jj',
           'professional_jk', 'professional_jl', 'professional_jm', 'professional_jn', 'professional_jo', 'professional_jp', 'professional_jq'] 

In [38]:
print(len(all_wellresults))

977


In [40]:
# 补充一些mislist
for mis in mislist:
    temp_results = []
    tag = mis.split("_")[0]
    misletter = mis.split("_")[1]
    if tag == 'patient':
        url = f'https://www.drugs.com/alpha/{misletter}.html'
    if tag == 'professional':
        url = f'https://www.drugs.com/alpha/{misletter}.html?pro=1'
    r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
    r.encoding = 'utf-8'
    html = r.text
    with open(f'D:/Intern_Project/9_drugs/{tag}_{misletter}列表页.html', 'w+', encoding='utf-8') as f:
         f.write(html)
    size = round(os.path.getsize(f'D:/Intern_Project/9_drugs/{tag}_{misletter}列表页.html') / float(1024),2)
    if  size<2.0:
        time.sleep(10)
        headers_pc, randomproxy, cookie_dict = changeparams()
        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
        with open(f'D:/Intern_Project/9_drugs/{tag}_{misletter}列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)
                
    selector = Selector(text=html)
    termUrl_list = selector.xpath(
        "//ul[@class='ddc-list-unstyled']/li/a")
    #print(len(termUrl_list))
    for termUrl in termUrl_list:
        temp_url = "https://www.drugs.com" + termUrl.xpath('./@href').extract()[0]
        temp_term = termUrl.xpath('./text()').extract()[0]
        # print(term)
        temp = {
            'term': temp_term,
            'url': temp_url
        }
        temp_results.append(temp)
    all_wellresults.extend(temp_results)

with open(f'D:/Intern_Project/9_drugs/术语链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(all_wellresults, indent=2, ensure_ascii=False))

In [41]:
print(len(all_wellresults))

1020


In [13]:
# 用来测试哪些页面能简单获取到
headers_pc, randomproxy, cookie_dict = changeparams()
            
requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
s = requests.session()
s.keep_alive = False # 关闭多余连接

# url = f'https://www.medicinenet.com/diseases_and_conditions/alpha_u.htm'
url = f'https://www.rxlist.com/supplements/alpha_a.htm'
# r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
try:  
    # 外网不能加params？？？
    r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
    
    
    #             encoding = cchardet.detect(r.content)['encoding']
    #             html = r.content.decode(encoding)
    r.encoding = 'utf-8'
    html = r.text
    with open('D:/Intern_Project/8_RxList/test列表页2.html', 'w+', encoding='utf-8') as f:
        f.write(html)
except Exception as e:
    print(e)

selector = Selector(text=html)
termUrl_list = selector.xpath(
    "//div[@id='AZ_container']/div[@class='AZ_results']/ul/li/a")
    
for termUrl in termUrl_list:
    url = termUrl.xpath('./@href').extract()[0]
    term = termUrl.xpath('./text()').extract()[0]
    # print(term)
    tmp = {
        'term': term,
        'url': url
    }

print(tmp)

{'term': 'Avocado Sugar Extract', 'url': 'https://www.rxlist.com/avocado_sugar_extract/supplements.htm'}
